## Stock Price Columns Explained

### 🔹 1. Open
The price of the stock at the moment the market opens in the morning.  
** Shows how the day begins compared to the previous day.

### 🔹 2. High
The highest price the stock reached during the entire trading day.  
** Shows the strongest buying pressure.

### 🔹 3. Low
The lowest price the stock reached during the trading day.  
** Shows the strongest selling pressure.

### 🔹 4. Close
The last price of the stock when the market closes.  
** Most important price for indicators like SMA, EMA, RSI, MACD.

### 🔹 5. Volume
The number of shares traded (bought + sold) during the day.  
** High volume = strong market interest; low volume = weak interest.

### 🔹 6. Adjusted Close (Adj Close)
The close price corrected for stock splits and dividends.  
** Best price for long-term analysis and charts.

### Why We Need These Columns
- Technical indicators use Open, High, Low, Close (especially Close).  
- Volume helps confirm trends.  
- Adjusted Close keeps historical data accurate.



Below are the most commonly used technical indicators

---

## 🔹 1. SMA – Simple Moving Average

###  What it is
SMA is the **average closing price** over a number of days.

It smooths the price so you can see the **overall trend**.

### Why it matters
- Price above SMA → trend is going UP  
- Price below SMA → trend is going DOWN  

###  Example (Using 3 days)
Imagine the closing prices for 3 days:

| Day | Price |
|-----|--------|
| 1   | 10     |
| 2   | 12     |
| 3   | 11     |

SMA-3 = (10 + 12 + 11) ÷ 3 = **11**

This means the “average” price for the last 3 days is 11.

---

## 🔹 2. EMA – Exponential Moving Average

### What it is
EMA is like SMA, but it gives **more weight to recent prices**.

So it reacts **faster** to changes.

###  Why it matters
- Faster trend detection  
- Used in MACD  

### Example  
We use a smoothing factor:

For EMA-3:

α = 2 ÷ (3 + 1) = **0.5**

Assume previous EMA = 11  
Today’s price = 13

EMA = (0.5 × 13) + (0.5 × 11)  
EMA = **12**

This shows EMA moves faster in the direction of the newest price.

---

## 🔹 3. RSI – Relative Strength Index

###  What it is
RSI shows whether the stock is:

- **Overbought** (70+) → may fall  
- **Oversold** (30−) → may rise  

It measures the **speed of price movements**.

### Example With Table

| Day | Price | Change | Up | Down |
|-----|--------|---------|------|--------|
| 1→2 | 10→12 | +2 | 2 | 0 |
| 2→3 | 12→11 | -1 | 0 | 1 |
| 3→4 | 11→13 | +2 | 2 | 0 |
| 4→5 | 13→15 | +2 | 2 | 0 |
| 5→6 | 15→14 | -1 | 0 | 1 |
| 6→7 | 14→16 | +2 | 2 | 0 |

Average Gain = (2 + 0 + 2 + 2 + 0 + 2) ÷ 6 = **1.33**  
Average Loss = (0 + 1 + 0 + 0 + 1 + 0) ÷ 6 = **0.33**

RS = 1.33 ÷ 0.33 = **4**

RSI = 100 − (100 ÷ (1 + 4))  
RSI = 100 − 20 = **80**

➡️ **RSI = 80 → Overbought**
Stock may drop soon.

---

## 🔹 4. MACD – Moving Average Convergence Divergence

###  What it is
MACD uses two EMAs to show:

- Trend direction  
- Trend strength  
- Buy/sell signals  

###  Components
- **MACD Line = EMA12 − EMA26**  
- **Signal Line = EMA9 of MACD**  
- **Histogram = MACD − Signal**

###  Example (Simplified)
Assume:

- EMA12 = 13  
- EMA26 = 12  

MACD Line = 13 − 12 = **1**

If Signal Line = 0.8:  
Histogram = 1 − 0.8 = **0.2**

###  How traders use it
- MACD crossing above Signal → **BUY**  
- MACD crossing below Signal → **SELL**

---


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# TA-Lib for indicators
import talib

# PyNance and PyFolio will be used later, we just import them now
import talib                      # technical indicators
import pynance as py              # from GitHub version
import quantstats as qs   

# Make plots a bit nicer
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 6)


ModuleNotFoundError: No module named 'quantstats'

Download TSLA data with yfinance

In [ ]:
# ============================================
# Download TSLA daily data for a year
# ============================================

ticker = "TSLA"
start_date = "2023-01-01"
end_date   = "2023-12-31"

df = yf.download(ticker, start=start_date, end=end_date, progress=False) 
df.head()


Flatten MultiIndex columns (if needed) and clean

In [ ]:
# ============================================
# Clean the columns (handle MultiIndex case)
# ============================================

# If columns are MultiIndex (like: ('Close', 'TSLA')), flatten them
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)  # keep only first level: Open, High, etc.

print("Cleaned columns:", df.columns.tolist())

# Optional: add a Ticker column (useful if using multiple tickers later)
df["Ticker"] = ticker

# Ensure index is datetime and sorted
df.index = pd.to_datetime(df.index)
df = df.sort_index()

df.head()


EDA: price and volume

In [5]:
# ============================================
# Quick look at price and volume
# ============================================

print(df.describe().T)

# Close price chart
df["Close"].plot(title="TSLA Close Price (2023)")
plt.ylabel("Price ($)")
plt.show()

# Volume chart
df["Volume"].plot(title="TSLA Daily Volume (2023)")
plt.ylabel("Shares traded")
plt.show()


NameError: name 'df' is not defined

Add TA-Lib indicators

In [ ]:
# ============================================
# Compute technical indicators with TA-Lib
# ============================================

# Use 'Adj Close' if available, otherwise 'Close'
if "Adj Close" in df.columns:
    price = df["Adj Close"]
else:
    price = df["Close"]

# --- Moving Averages ---
df["SMA_20"] = talib.SMA(price, timeperiod=20)
df["SMA_50"] = talib.SMA(price, timeperiod=50)
df["EMA_20"] = talib.EMA(price, timeperiod=20)

# --- RSI ---
df["RSI_14"] = talib.RSI(price, timeperiod=14)

# --- MACD ---
df["MACD"], df["MACD_signal"], df["MACD_hist"] = talib.MACD(
    price, fastperiod=12, slowperiod=26, signalperiod=9
)


df.tail()


Quick Reading 

Price action: TSLA climbed from 252.54 → 261.44 (Dec 27 peak), then pulled back to 248.48 by Dec 29.

Trend: SMA_20 (~244–247) stays well above SMA_50 (~232–233) → bullish medium-term trend.

Short-term strength: EMA_20 is slightly above SMA_20 and rising → recent prices are stronger than older ones.

Momentum (RSI & MACD):
RSI > 70 → Overbought (price may fall)

RSI < 30 → Oversold (price may rise)

RSI 50–70 → Bullish momentum

RSI 30–50 → Bearish/weak momentum

RSI_14 moves from 58 → 64, then cools to 52 → momentum was strong mid-week, then weakened.

> 0 → MACD above signal → bullish momentum
< 0 → MACD below signal → bearish momentum / loss of steam

MACD stays positive (~5–6) all week → overall bullish bias, but the drop from 6.03 to 5.04 shows momentum fading at the end.

Plot price + SMA

In [ ]:
# ============================================
# Price + SMA + Bollinger Bands
# ============================================

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(df.index, price, label="Price", color="black")
ax.plot(df.index, df["SMA_20"], label="SMA 20", linestyle="--")
ax.plot(df.index, df["SMA_50"], label="SMA 50", linestyle="-.")


ax.set_title("TSLA – Price, Moving Averages")
ax.set_xlabel("Date")
ax.set_ylabel("Price ($)")
ax.legend()
plt.show()


Plot RSI

In [ ]:
# ============================================
# RSI – Overbought / Oversold
# ============================================

fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(df.index, df["RSI_14"], label="RSI 14")
ax.axhline(70, color="red", linestyle="--", label="Overbought (70)")
ax.axhline(30, color="green", linestyle="--", label="Oversold (30)")

ax.set_title("TSLA – RSI(14)")
ax.set_ylabel("RSI value")
ax.legend()
plt.show()


Plot MACD

In [ ]:
# ============================================
# MACD – Trend & momentum
# ============================================

fig, ax = plt.subplots(figsize=(12, 4))

ax.plot(df.index, df["MACD"], label="MACD")
ax.plot(df.index, df["MACD_signal"], label="Signal")
ax.bar(df.index, df["MACD_hist"], label="Histogram")

ax.set_title("TSLA – MACD")
ax.set_ylabel("MACD value")
ax.legend()
plt.show()


MACD vs Signal Crossovers
MACD crossing above Signal → bullish momentum starts
MACD crossing below Signal → bearish momentum starts

Histogram (Blue Bars)
Histogram > 0 → bullish momentum accelerating
Histogram < 0 → bearish momentum accelerating
Increasing bar height = momentum building
Decreasing bar height = momentum fading


Portfolio optimisation using PyNance

In [ ]:
# ============================================
# PyNance – Basic portfolio optimisation demo
# ============================================

from pynance import portfolio_optimizer as po

TICKERS = ["AAPL", "MSFT", "META", "NVDA","TSLA"]

# Create the portfolio calculations object
portfolio = po.PortfolioCalculations(TICKERS)

print("=== Max Sharpe Portfolio (risk/return) ===")
print(portfolio.max_sharpe_portfolio("rr"))

print("\n=== Max Sharpe Portfolio weights ===")
print(portfolio.max_sharpe_portfolio("df").head())

print("\n=== Min Variance Portfolio (risk/return) ===")
print(portfolio.min_var_portfolio("rr"))

print("\n=== Min Variance Portfolio weights ===")
print(portfolio.min_var_portfolio("df").head())
